---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

def blight_model():
    # variables (to be changed if needed)
    train = 'train.csv'
    test = 'test.csv'
    addresses = 'addresses.csv'
    latlons = 'latlons.csv'
    # remove non-existing features from training data or both training and testing data    
    features_to_remove_from_train_only = [
        'payment_amount',
        'payment_date',
        'payment_status',
        'balance_due',
        'collection_status',
        'compliance_detail'
    ]
    features_to_remove_general = [
#         'agency_name',
        'inspector_name',
        'violator_name',
        'violation_street_number',
        'violation_street_name',
        'violation_zip_code',
        'mailing_address_str_number',
        'mailing_address_str_name',
        'city',
        'state',
        'zip_code',
        'non_us_str_code',
        'country',
        'ticket_issued_date',
        'hearing_date',
        'violation_code',
        'violation_description',
#         'disposition',
        'grafitti_status',
        'state_fee',
        'admin_fee'
    ]


    # https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
    df_train = pd.read_csv(train, encoding="latin-1", low_memory=False)
    df_test = pd.read_csv(test)
    df_addresses = pd.read_csv(addresses)
    df_latlons = pd.read_csv(latlons)

    # remove Null if the violator was found not responsible
    df_train_filtered = df_train[df_train['compliance'].notnull()]
    df_train_filtered = df_train_filtered[df_train_filtered['hearing_date'].notnull()]
#     df_test_filtered =  df_test[df_test['hearing_date'].notnull()]

    # hot one encode agency_name and disposition
    df_train_filtered = pd.get_dummies(df_train_filtered, columns=['agency_name'])
    df_train_filtered = pd.get_dummies(df_train_filtered, columns=['disposition'])    
    df_test = pd.get_dummies(df_test, columns=['agency_name'])
    df_test = pd.get_dummies(df_test, columns=['disposition'])
    
    # add non existing columns will default value of 0
    df_test['agency_name_Health Department'] = 0
    df_test['agency_name_Neighborhood City Halls'] = 0
    df_train_filtered['disposition_Responsible (Fine Waived) by Admis'] = 0
    df_train_filtered['disposition_Responsible - Compl/Adj by Default'] = 0
    df_train_filtered['disposition_Responsible - Compl/Adj by Determi'] = 0
    df_train_filtered['disposition_Responsible by Dismissal'] = 0
    
    # fix dates
    df_train_filtered['ticket_issued_date'] = pd.to_datetime(df_train_filtered['ticket_issued_date'])
    df_test['ticket_issued_date'] = pd.to_datetime(df_test['ticket_issued_date'])
    df_train_filtered['hearing_date'] = pd.to_datetime(df_train_filtered['hearing_date'])
    df_test['hearing_date'] = pd.to_datetime(df_test['hearing_date'])
    df_train_filtered['days_between_ticket'] = df_train_filtered['hearing_date'] - df_train_filtered['ticket_issued_date']
    df_test['days_between_ticket'] = df_test['hearing_date'] - df_test['ticket_issued_date']
    df_train_filtered['days_between_ticket'] = df_train_filtered['days_between_ticket'].dt.days
    df_test['days_between_ticket'] = df_test['days_between_ticket'].dt.days
    df_test['days_between_ticket'].fillna(0, inplace=True)
    df_train_filtered['days_between_ticket'].fillna(0, inplace=True)

    # drop features from training data
    df_train_filtered = df_train_filtered.drop(features_to_remove_from_train_only, axis=1)
    df_train_filtered = df_train_filtered.drop(features_to_remove_general, axis=1)
    df_test_filtered = df_test.drop(features_to_remove_general, axis=1)
    
    # join addresses and latlons together based on common column of address
    # https://stackoverflow.com/questions/45068584/merge-two-csv-files-into-one-with-pandas-by-id
    df_locations = pd.merge(df_addresses, df_latlons, how='inner', on=['address', 'address'])

    # join addresses to train and test dataframe
    df_train_filtered = pd.merge(df_train_filtered, df_locations, how='inner', on=['ticket_id', 'ticket_id'])
    df_test_filtered = pd.merge(df_test_filtered, df_locations, how='inner', on=['ticket_id', 'ticket_id'])

    df_train_filtered = df_train_filtered.drop('address', axis=1)
    df_test_filtered = df_test_filtered.drop('address', axis=1)
    
    # fill NaN values
    df_train_filtered.lat.fillna(method='pad', inplace=True)
    df_train_filtered.lon.fillna(method='pad', inplace=True)
    df_test_filtered.lat.fillna(method='pad', inplace=True)
    df_test_filtered.lon.fillna(method='pad', inplace=True)
    df_train_filtered.lat.fillna(method='bfill', inplace=True)
    df_train_filtered.lon.fillna(method='bfill', inplace=True)
    df_test_filtered.lat.fillna(method='bfill', inplace=True)
    df_test_filtered.lon.fillna(method='bfill', inplace=True)
    
#     df_train_filtered = df_train_filtered.reindex(sorted(df_train_filtered.columns), axis=1)
#     df_test_filtered = df_test_filtered.reindex(sorted(df_test_filtered.columns), axis=1)

#     print(df_train_filtered.shape)
#     print()
#     print(df_test_filtered.shape)
        
    y = df_train_filtered.compliance
    X = df_train_filtered.drop('compliance', axis=1)
    X_test = df_test_filtered 
    
#     print(X_test.isnull().any(axis=1).value_counts())
    
    # Split the dataset in two equal parts
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html
    # http://scikit-learn.org/stable/modules/grid_search.html
    # Set the parameters by cross-validation
    # two grids should be explored linear and RBF
    # 'svm__gamma': [0.0001, 0.001, 0.01, 0.05, 0.1, 1, 10, 100, 1000],
    # svm too slow, removed testing only gradient boost and randomforest

    params1 = [
        {
            'svm__kernel': ['linear'],
            'svm__gamma': [0.0001, 0.001, 0.01, 0.05, 0.1, 1, 10, 100, 1000],
            'svm__C': [1, 10, 100, 1000]
        }, {
            'svm__kernel': ['rbf'],
            'svm__gamma': [0.0001, 0.001, 0.01, 0.05, 0.1, 1, 10, 100, 1000],
            'svm__C': [1, 10, 100, 1000]
        }
    ]    
    params2 = [
        {
            'gb__n_estimators': [10, 20, 50, 100],
            'gb__criterion': ['friedman_mse', 'mse', 'mae'],
            'gb__max_features': ['auto', 'sqrt', None],
            'gb__max_depth': [3, 4, 5, 6, 7]
        }
    ]
    params3 = [
        {
            'rf__n_estimators': [10, 20, 50, 100, 150],
            'rf__max_features' : ['auto', 'sqrt', 'log2'],
            'rf__min_samples_split' : [2, 5, 10],
            'rf__min_samples_leaf' : [1, 2, 4],
            'rf__bootstrap': [True, False],
        } 
    ]

    # assignment only requires roc_auc
    scores = ['roc_auc']

#     for score in scores:
#         print("# Tuning hyper-parameters for %s" % score)
#         print()

# #         pipe = Pipeline(steps=[
# #             ('std', StandardScaler()), 
# #             ('svm', SVC())
# #         ])
# #         estimator = GridSearchCV(pipe, params1, scoring='%s'%score, n_jobs=-1)
# #         estimator.fit(X_train, y_train)

# #         print("Best parameters set found on development set: ")
# #         print()
# #         print(estimator.best_params_)
# #         print()
# #         print('Grid best score (AUC): ', estimator.best_score_)
# #         print()
        
# #         pipe = Pipeline(steps=[
# #             ('max', MaxAbsScaler()), 
# #             ('svm', SVC())
# #         ])
# #         estimator = GridSearchCV(pipe, params1, scoring='%s'%score, n_jobs=-1)
# #         estimator.fit(X_train, y_train)

# #         print("Best parameters set found on development set: ")
# #         print()
# #         print(estimator.best_params_)
# #         print()
# #         print('Grid best score (AUC): ', estimator.best_score_)
# #         print()
        
# #         pipe = Pipeline(steps=[
# #             ('minmax', MinMaxScaler()), 
# #             ('svm', SVC())
# #         ])
# #         estimator = GridSearchCV(pipe, params1, scoring='%s'%score, n_jobs=-1)
# #         estimator.fit(X_train, y_train)

# #         print("Best parameters set found on development set: ")
# #         print()
# #         print(estimator.best_params_)
# #         print()
# #         print('Grid best score (AUC): ', estimator.best_score_)
# #         print()
        
#         pipe = Pipeline(steps=[
#             ('minmax', MinMaxScaler()), 
#             ('gb', GradientBoostingClassifier())
#         ])
#         estimator = GridSearchCV(pipe, params2, cv=5, scoring='%s'%score, n_jobs=-1)
# #         estimator.fit(X_train, y_train)
#         estimator.fit(X, y)

#         print("Best parameters (GradientBoostingClassifier) set found on development set: ")
#         print()
#         print(estimator.best_params_)
#         print()
#         print('Grid best score (AUC): ', estimator.best_score_)
#         print()
        
#         pipe = Pipeline(steps=[
#             ('minmax', MinMaxScaler()), 
#             ('rf', RandomForestClassifier())
#         ])
#         estimator2 = GridSearchCV(pipe, params3, scoring='%s'%score, n_jobs=-1)
# #         estimator2.fit(X_train, y_train)
#         estimator2.fit(X, y)

#         print("Best parameters (RandomForestClassifier) set found on development set: ")
#         print()
#         print(estimator2.best_params_)
#         print()
#         print('Grid best score (AUC): ', estimator2.best_score_)
#         print()
    
    # remove gb__ prefix from best_params_
#     new_gb_best_params_ = dict((key.replace('gb__', ''), value) for (key, value) in estimator.best_params_.items())
    
    new_gb_best_params_ = {'criterion': 'friedman_mse', 'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 10}
    
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_test_scaled = scaler.transform(X_test)
    estimator3 = GradientBoostingClassifier(**new_gb_best_params_)
    estimator3.fit(X_scaled, y)
    yhat_prob = estimator3.predict_proba(X_test_scaled)[:,1]
    results = pd.DataFrame(columns=['ticket_id','compliance'])
    results['ticket_id'] = df_test_filtered.ticket_id
    results['compliance'] = yhat_prob
    results.set_index('ticket_id', inplace=True)
    
    return results

In [37]:
blight_model()

,compliance
ticket_id,
284932,0.206498
285362,0.092639
285361,0.141300
285338,0.162379
285346,0.162379
285345,0.162379
285347,0.162379
285342,0.537882
285530,0.105829
